In [1]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as skmetrics
from sklearn.metrics import average_precision_score
from sklearn.model_selection import *
from sklearn.svm import SVC as SVM
from keras.models import *
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras.regularizers import *
from keras import metrics
from keras import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SameenShaukat\

In [2]:
path_save = "/local_mimic/save"
path_tables = "/local_mimic/tables"
path_views = "/local_mimic/views"
path_chkpt = "/local_mimic/save/checkpoint"

In [3]:
labtest = pd.read_csv(path_views + '/labtest_dataset.csv')

In [4]:
labtest.head()

,subject_id,hadm_id,hct_med,hct_lowest,hct_highest,hematocrit_first,wbc_first,hgb_first,platelet_first,sodium_first,...,ldh_first,bilirubin_first,alp_first,albumin_first,tropt_first,ck_first,ntbnp_first,lactate_first,hct_abnormal_flg,potassium_abnormal_flg
0,10272,104675,29.95,27.8,36.7,28.7,15.5,8.3,343.0,138.0,...,NaN,0.5,273.0,NaN,NaN,13.0,NaN,1.0,1,0.0
1,9849,156939,34.50,33.1,35.9,33.1,30.3,11.0,108.0,133.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1,1,0.0
2,22520,101952,35.40,31.0,36.0,35.0,5.8,11.5,317.0,139.0,...,NaN,0.3,95.0,3.2,0.76,121.0,NaN,NaN,1,0.0
3,2560,126863,48.70,48.7,48.7,48.7,13.0,15.6,334.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
4,297,107224,52.10,52.1,52.1,52.1,14.9,17.0,342.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0


In [5]:
features = labtest[labtest.columns[1:29]]

In [6]:
labels = labtest['potassium_abnormal_flg']

In [7]:
features.head(5)

,hadm_id,hct_med,hct_lowest,hct_highest,hematocrit_first,wbc_first,hgb_first,platelet_first,sodium_first,potassium_first,...,ast_first,alt_first,ldh_first,bilirubin_first,alp_first,albumin_first,tropt_first,ck_first,ntbnp_first,lactate_first
0,104675,29.95,27.8,36.7,28.7,15.5,8.3,343.0,138.0,4.5,...,15.0,6.0,NaN,0.5,273.0,NaN,NaN,13.0,NaN,1.0
1,156939,34.50,33.1,35.9,33.1,30.3,11.0,108.0,133.0,4.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1
2,101952,35.40,31.0,36.0,35.0,5.8,11.5,317.0,139.0,4.0,...,26.0,18.0,NaN,0.3,95.0,3.2,0.76,121.0,NaN,NaN
3,126863,48.70,48.7,48.7,48.7,13.0,15.6,334.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,107224,52.10,52.1,52.1,52.1,14.9,17.0,342.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
features = features.apply(lambda x: x.fillna(round(x.mean(),2),axis=0))

In [9]:
labels = labels.fillna(0)

Regression Preparation

In [10]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean',  verbose=0, copy=True)
imputer.fit(features)
features = imputer.transform(features)
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)

In [13]:
penalty = 'l2'
C=0.0001
logreg = LogisticRegression(penalty=penalty, C=C, random_state=42)
logreg.fit(X_train, y_train)

LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
filename = path_chkpt + '/PotassiumLabtest/lr_best_model'
pickle.dump(logreg, open(filename, 'wb'))

In [15]:
prediction = logreg.predict_proba(X_test)[:, 1]

In [16]:
y_pred=logreg.predict(X_test)

In [17]:
cnf_matrix = skmetrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[9916,    6],
       [1523,   21]], dtype=int64)

In [18]:
print("Accuracy:",skmetrics.accuracy_score(y_test, y_pred))
print("Precision:",skmetrics.precision_score(y_test, y_pred))
print("Recall:",skmetrics.recall_score(y_test, y_pred))

Accuracy: 0.866649223792081
Precision: 0.7777777777777778
Recall: 0.013601036269430052


In [19]:
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = skmetrics.roc_curve(y_test,  y_pred_proba)
auc = skmetrics.roc_auc_score(y_test, y_pred_proba)

In [20]:
print("AUC:",auc)

AUC: 0.6093787370505487


In [21]:
average_precision_score(y_test, y_pred_proba)

0.3863233532948928

SVM Preparation

In [22]:
svm_model = SVM(kernel = 'linear', probability=True)

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
filename = path_chkpt + '/PotassiumLabtest/svm_best_model'
pickle.dump(svm_model, open(filename, 'wb'))

In [ ]:
prediction_probabilities = svm_model.predict_proba(X_test)[::,1]

In [ ]:
label_prediction =svm_model.predict(X_test)

In [27]:
cnf_matrix = skmetrics.confusion_matrix(y_test, label_prediction)
cnf_matrix

array([[9922,    0],
       [1544,    0]], dtype=int64)

In [28]:
print("Accuracy:",skmetrics.accuracy_score(y_test, label_prediction))
print("Precision:",skmetrics.precision_score(y_test, label_prediction))
print("Recall:",skmetrics.recall_score(y_test, label_prediction))

Accuracy: 0.8653410081981511
Precision: 0.0
Recall: 0.0


C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction_probabilities)
auc = skmetrics.roc_auc_score(y_test, prediction_probabilities)

In [30]:
print("AUC:",auc)

AUC: 0.5698707692018469


In [31]:
average_precision_score(y_test, prediction_probabilities)

0.4715286839988409

LSTM Preparation

In [32]:
filename = path_chkpt + '/PotassiumLabtest/lstm_best_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [33]:
X = features
y = labels.to_numpy()
input_shape = (X.shape[1], X.shape[-1])

In [34]:
def lstm_model(input_shape, hidden = 256, targets = 1, learn_rate = 1e-4):
    model = Sequential()
    #model.add(Bidirectional(LSTM(hidden), merge_mode = 'concat'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(targets))
    model.add(Activation ('sigmoid'))
    model.compile(loss='binary_crossentropy', 
    optimizer=Adam(lr=learn_rate, beta_1 =.5 ), metrics=['accuracy'])
    return (model)

In [35]:
lstm_model = lstm_model(input_shape=input_shape, targets=1, hidden = 256, learn_rate=1e-4)

In [36]:
lstm_model.fit(x = X, y= y, validation_split=.2, epochs = 100, verbose=2, batch_size=32, callbacks=[checkpoint])



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 45862 samples, validate on 11466 samples
Epoch 1/100
 - 2s - loss: 0.7040 - acc: 0.5925 - val_loss: 0.6354 - val_acc: 0.7180

Epoch 00001: val_loss improved from -inf to 0.63540, saving model to C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save/checkpoint/PotassiumLabtest/lstm_best_model
Epoch 2/100
 - 1s - loss: 0.6250 - acc: 0.7110 - val_loss: 0.5766 - val_acc: 0.8113

Epoch 00002: val_loss did not improve from 0.63540
Epoch 3/100
 - 1s - loss: 0.5725 - acc: 0.7829 - val_loss: 0.5333 - val_acc: 0.8500

Epoch 00003: val_loss did not improve from 0.63540
Epoch 4/100
 - 2s - loss: 0.5312 - acc: 0.8247 - val_loss: 0.4989 - val_acc: 0.8629

Epoch 00004: val_loss did not improve from 0.63540
Epoch 5/100
 - 1s - loss: 0.5010 - acc: 0.8416 - val_loss: 0


Epoch 00048: val_loss did not improve from 0.63540
Epoch 49/100
 - 1s - loss: 0.3811 - acc: 0.8665 - val_loss: 0.3585 - val_acc: 0.8772

Epoch 00049: val_loss did not improve from 0.63540
Epoch 50/100
 - 1s - loss: 0.3815 - acc: 0.8664 - val_loss: 0.3585 - val_acc: 0.8772

Epoch 00050: val_loss did not improve from 0.63540
Epoch 51/100
 - 2s - loss: 0.3809 - acc: 0.8665 - val_loss: 0.3584 - val_acc: 0.8772

Epoch 00051: val_loss did not improve from 0.63540
Epoch 52/100
 - 1s - loss: 0.3817 - acc: 0.8666 - val_loss: 0.3585 - val_acc: 0.8772

Epoch 00052: val_loss did not improve from 0.63540
Epoch 53/100
 - 1s - loss: 0.3811 - acc: 0.8665 - val_loss: 0.3585 - val_acc: 0.8772

Epoch 00053: val_loss did not improve from 0.63540
Epoch 54/100
 - 1s - loss: 0.3810 - acc: 0.8664 - val_loss: 0.3584 - val_acc: 0.8772

Epoch 00054: val_loss did not improve from 0.63540
Epoch 55/100
 - 2s - loss: 0.3805 - acc: 0.8666 - val_loss: 0.3584 - val_acc: 0.8772

Epoch 00055: val_loss did not improve fr

In [37]:
prediction = lstm_model.predict_proba(X_test,batch_size=32)

In [38]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction)
auc = skmetrics.roc_auc_score(y_test, prediction)

In [39]:
print("AUC:",auc)

AUC: 0.6145505539059587


In [40]:
average_precision_score(y_test, prediction)

0.304370515291674

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

TCN Preparation

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.preprocessing import sequence
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.model_selection import *
from sklearn import metrics as skmetrics
from sklearn.metrics import average_precision_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import tensorflow as tf
from tcn import TCN


C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\SameenShaukat\anaconda3\envs\myenv\lib\s

In [2]:
path_save = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save"
path_tables = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/tables"
path_views = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/views"
path_chkpt = "C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save/checkpoint"

In [3]:
labtest = pd.read_csv(path_views + '/labtest_dataset.csv')

In [4]:
features = labtest[labtest.columns[1:29]]

In [5]:
labels = labtest['potassium_abnormal_flg']

In [6]:
features = features.apply(lambda x: x.fillna(round(x.mean(),2),axis=0))

In [7]:
labels = labels.fillna(0)

In [8]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean',  verbose=0, copy=True)
imputer.fit(features)
features = imputer.transform(features)
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)

In [10]:
batch_size = 32
maxlen = 24
max_features = 45862

print('Loading data...')
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')


print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Loading data...
45862 train sequences
11466 test sequences
x_train shape: (45862, 28)
x_test shape: (11466, 28)


In [11]:
X_train = X_train.reshape(-1,28, 1)
X_test = X_test.reshape(-1,28, 1)
max_features = 45862

In [12]:
tcn_layer = TCN(input_shape=(28, 1))
model = Sequential()
model.add(tcn_layer)
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

metrics = ['accuracy',
          tf.keras.metrics.AUC(),
          tf.keras.metrics.Precision()]

model.compile('adam', 'binary_crossentropy', metrics=metrics)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tcn (TCN)                    (None, 64)                136256    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 136,321
Trainable params: 136,321
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
filename = path_chkpt + '/PotassiumLabtest/TCN_best_model'
checkpoint = ModelCheckpoint(filename, monitor='val_auc',verbose=1, save_best_only=True, mode='max')
print('Train...')
model.fit(
    X_train, y_train,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint],
    epochs=10
)

Train...
Train on 45862 samples, validate on 11466 samples
Epoch 1/10
45824/45862 [============================>.] - ETA: 0s - loss: 0.0862 - acc: 0.9683 - auc: 0.9903 - precision: 0.8901
Epoch 00001: val_auc improved from -inf to 0.99319, saving model to C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save/checkpoint/PotassiumLabtest/TCN_best_model
45862/45862 [==============================] - 54s 1ms/sample - loss: 0.0862 - acc: 0.9683 - auc: 0.9903 - precision: 0.8900 - val_loss: 0.0911 - val_acc: 0.9631 - val_auc: 0.9932 - val_precision: 0.8061
Epoch 2/10
45824/45862 [============================>.] - ETA: 0s - loss: 0.0759 - acc: 0.9683 - auc: 0.9921 - precision: 0.8935
Epoch 00002: val_auc improved from 0.99319 to 0.99573, saving model to C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save/checkpoint/PotassiumLabtest/TCN_best_model
45862/45862 [==============================] - 50s 1ms/sample - loss: 0.0758 - acc: 0.9684 - auc: 0.9921 - preci

MRM Preparation

In [34]:
def mrm_model(input_shape, hidden = 256, targets = 1, learn_rate = 1e-4):
    model = Sequential()
    model.add(Dropout(0.5))
    model.add(Dense(targets))
    model.add(Activation ('sigmoid'))
    model.compile(loss='binary_crossentropy', 
    optimizer=Adam(lr=learn_rate, beta_1 =.5 ), metrics=['accuracy'])
    return (model)

In [35]:
filename = path_chkpt + '/PotassiumLabtest/mrm_best_model'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='max')

In [36]:
X = features
y = labels.to_numpy()
input_shape = (X.shape[1], X.shape[-1])

In [37]:
mrm_model = mrm_model(input_shape=input_shape, targets=1, hidden = 256, learn_rate=1e-4)

In [38]:
mrm_model.fit(x = X, y= y, validation_split=.2, epochs = 10, verbose=2, batch_size=64, callbacks=[checkpoint])

Train on 45862 samples, validate on 11466 samples
Epoch 1/10
 - 1s - loss: 0.8726 - acc: 0.5507 - val_loss: 0.7478 - val_acc: 0.5468

Epoch 00001: val_loss improved from -inf to 0.74784, saving model to C:/Users/SameenShaukat/Downloads/Project/DL4H-Project/local_mimic/save/checkpoint/PotassiumLabtest/mrm_best_model
Epoch 2/10
 - 1s - loss: 0.7933 - acc: 0.5841 - val_loss: 0.6915 - val_acc: 0.6057

Epoch 00002: val_loss did not improve from 0.74784
Epoch 3/10
 - 1s - loss: 0.7328 - acc: 0.6172 - val_loss: 0.6483 - val_acc: 0.6790

Epoch 00003: val_loss did not improve from 0.74784
Epoch 4/10
 - 1s - loss: 0.6838 - acc: 0.6626 - val_loss: 0.6137 - val_acc: 0.7638

Epoch 00004: val_loss did not improve from 0.74784
Epoch 5/10
 - 1s - loss: 0.6396 - acc: 0.7113 - val_loss: 0.5850 - val_acc: 0.8053

Epoch 00005: val_loss did not improve from 0.74784
Epoch 6/10
 - 1s - loss: 0.6039 - acc: 0.7607 - val_loss: 0.5602 - val_acc: 0.8400

Epoch 00006: val_loss did not improve from 0.74784
Epoch 7/

In [39]:
prediction = mrm_model.predict_proba(X_test,batch_size=64)

In [40]:
fpr, tpr, _ = skmetrics.roc_curve(y_test,  prediction)
auc = skmetrics.roc_auc_score(y_test, prediction)

In [41]:
auc, average_precision_score(y_test, prediction)

(0.6127932589221837, 0.32266361040193026)